<a href="https://colab.research.google.com/github/tahniyath123/project1/blob/main/latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### The brief overview of this project is, we are going to use the hugging face encoder decoder model, which accepts the image as an input and run through the encoder decoder model to caption the image.

In [1]:
!pip install transformers
!pip install dash
!pip install gTTS
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 37.2 MB/s eta 0:00:00


In [2]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

In [3]:
model=VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

In [4]:
feature_extractor=ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer=AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [5]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [6]:
max_length = 16
num_beams = 4

gen_kwargs = {"max_length":max_length, "num_beams":num_beams}

In [7]:
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image=Image.open(image_path)
    if i_image.mode != 'RGB':
      i_image = i_image.convert(mode='RGB')
    images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors='pt').pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)
    preds=tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds=[pred.strip() for pred in preds]
    return preds

In [8]:
#from tensorflow.python.ops.string_ops import encode_base64
from dash import Dash, dcc, html, Input, Output, State, callback
from base64 import decodebytes
from base64 import encodebytes
import datetime
import os
from gtts import gTTS
from IPython.display import Audio
from IPython.display import display
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout = html.Div(style={'backgroundColor': colors['background'],'justify-content': 'center'}, children=[
        html.H4(
        children='Visio Voice:',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'justify-content': 'center'
        }
    ),
    html.Div(children='EMPOWERING THE VISUALLY IMPARED', style={
        'textAlign': 'center',
        'color': colors['text']
    }),
    #html.Hr(),
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select File')
        ]),
        style={
            #'position':'inherit',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px 100px 10px 100px',
            'color': colors['text']
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Br(),
    html.Div(id='output-image-upload'),
])

def parse_contents(contents,fname,verdict,sound):
    return html.Div([
        #html.H5(fname),
        #html.H6(datetime.datetime.fromtimestamp(date)),

        # HTML images accept base64 encoded strings in the same format
        # that is supplied by the upload
        html.Img(src=contents,style={'display': 'block','width': '80%','margin': 'auto'}),
        html.H6(verdict,style={'text-align': 'center','color': '#7FDBFF'}),
        html.Audio(id='audioplayer', src='data:audio/mpeg;base64,{}'.format(sound.decode()),
               controls=True,
               autoPlay=True, style={"width": "50%","margin":"0 auto", "display":"block"}
               ),
        #html.Hr(),
        html.Br()
        #html.Div('Raw Content'),
        # html.Pre(contents[0:200] + '...', style={
        #     'whiteSpace': 'pre-wrap',
        #     'wordBreak': 'break-all'
        # })
    ],style={"justify-content": "center","align-items": "center"})

@callback(Output('output-image-upload', 'children'),
              Input('upload-image', 'contents'),
              State('upload-image', 'filename'))
def update_output(images,filenames):
    if not images:
        return
    children=[]
    for image_str,fname in zip(images,filenames):
        image = image_str.split(',')[1]
        data = decodebytes(image.encode('ascii'))
        with open(fname, "wb") as f:
            f.write(data)
        verdict=predict_step([fname])
        myobj = gTTS(text=str(verdict), lang='en', slow=False)
        myobj.save(str(fname).split('.')[0]+".mp3")
        # Playing the converted file
        encoded_sound = encodebytes(open(str(fname).split('.')[0]+".mp3", 'rb').read())
        #os.system("mpg321 "+str(fname).split('.')[0]+".mp3")
        children.append(parse_contents(image_str,fname,verdict,encoded_sound))
    return children


In [9]:
app.run_server(debug=True,mode='inline')

<IPython.core.display.Javascript object>

In [10]:
from pyngrok import ngrok
# Setting an auth token allows us to open multiple
# tunnels at the same time
#ngrok.set_auth_token("2VcvEGTU09Tn1bvQFCpotIfXl2U_3JA5YssZWSK3JNF7KqB9t")
ngrok.set_auth_token("2VhC4V1tvjyZUHC48rDO4lbLifJ_3zyhPRBdvT55xjaN3ig4m")
# <NgrokTunnel: "https://<public_sub1>.ngrok.io" -> "http://localhost:80">
#ngrok_tunnel1 = ngrok.connect()
# <NgrokTunnel: "https://<public_sub2>.ngrok.io" -> "http://localhost:8000">
ngrok.kill()
ngrok_tunnel2 = ngrok.connect(8050)
print(ngrok_tunnel2)

NgrokTunnel: "https://1416-34-16-191-3.ngrok-free.app" -> "http://localhost:8050"
